In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import math
import copy

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

dm.Utilities.set_compute_backend('torch')

In [ ]:
def generate_square(width_density, center, width):
    points = torch.zeros(4*width_density, 2)
    points[0:width_density, 0] = torch.linspace(0., 1., width_density)
    points[width_density:2*width_density, 1] = torch.linspace(0., 1., width_density)
    points[width_density:2*width_density, 0] = 1.
    points[2*width_density:3*width_density, 0] = torch.linspace(1., 0., width_density)
    points[2*width_density:3*width_density, 1] = 1.
    points[3*width_density:4*width_density, 1] = torch.linspace(1., 0., width_density)
    return width*(points - torch.tensor([0.5, 0.5])) + center

width_density = 100
silent_points = generate_square(width_density, torch.tensor([0., 0.]), 0.5)
silent_points = silent_points*torch.tensor([1., 3.])
aabb = dm.Utilities.AABB.build_from_points(silent_points).scale(0.98)

implicit1_points = aabb.fill_uniform_density(10.)
implicit1_points = implicit1_points - torch.mean(implicit1_points, dim=0)
implicit1_cotan = torch.zeros_like(implicit1_points)
implicit1_cotan[0, 0] = 50.
implicit1_R = torch.eye(2).repeat(implicit1_points.shape[0], 1, 1)
implicit1_cotan_R = torch.zeros_like(implicit1_R)

In [ ]:
def shoot(C, moment_scale):
    silent = dm.DeformationModules.SilentLandmarks(2, silent_points.shape[0], gd=silent_points.clone().requires_grad_())

    implicit1 = dm.DeformationModules.ImplicitModule1(2, implicit1_points.shape[0], 2., C, nu=0.01, gd=(implicit1_points.clone().requires_grad_(), implicit1_R.clone().requires_grad_()), cotan=(moment_scale*implicit1_cotan.clone().requires_grad_(), implicit1_cotan_R.clone().requires_grad_()))

    dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, implicit1]), 100, 'euler')

    return silent.manifold.gd.detach(), implicit1.manifold.gd[0].detach()

def shoot_and_plot(C, moment_scale):
    out_silent, out_implicit1 = shoot(C, moment_scale)

    aabb = dm.Utilities.AABB.build_from_points(torch.cat([out_silent, silent_points])).squared()

    plt.subplot(1, 3, 1)
    plt.plot(silent_points[:, 0].numpy(), silent_points[:, 1].numpy())
    plt.plot(implicit1_points[:, 0].numpy(), implicit1_points[:, 1].numpy(), '.')
    plt.axis(aabb.get_list())

    plt.subplot(1, 3, 2)
    plt.plot(out_silent[:, 0].numpy(), out_silent[:, 1].numpy())
    plt.plot(out_implicit1[:, 0].numpy(), out_implicit1[:, 1].numpy(), '.')
    plt.axis(aabb.get_list())

    ax = plt.subplot(1, 3, 3)
    plt.plot(implicit1_points[:, 0].numpy(), implicit1_points[:, 1].numpy(), '.')
    dm.Utilities.plot_C_arrow(ax, implicit1_points, C, color='red', mutation_scale=10., scale=0.2)
    plt.axis(aabb.get_list())

    plt.show()

In [ ]:
C = torch.ones(implicit1_points.shape[0], 2, 1)
#C[:, 1, 0] = 2.*torch.abs(implicit1_points[:, 0] - torch.mean(implicit1_points, dim=0)[0] + aabb.width)/aabb.width
C[:, 1] = -C[:, 1]

shoot_and_plot(C, -2)

In [ ]:
C = torch.zeros(implicit1_points.shape[0], 2, 1)
C[int(implicit1_points.shape[0]/2):, 1, 0] = 1.

shoot_and_plot(C, 3.)

In [ ]:
%matplotlib qt5

C = torch.zeros(implicit1_points.shape[0], 2, 1)
C[:, 0] = ((5.*implicit1_points[:, 1])**2).unsqueeze(1)

shoot_and_plot(C, -110.)